# Multicollinearity

In [8]:
# Import necessary libaries and data
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

advert = pd.read_csv('advertising.csv')

**Multicollinearity** is defined by two or more predictors in a regression model having moderate or high correlation. It is the reason for the sub-optimal performance of the final model in the previous step.

As shown in the below correlation matrix, there is a significant correlation of 0.354 between radio and newspaper:

In [2]:
advert.corr()

,TV,Radio,Newspaper,Sales
TV,1.000000,0.054809,0.056648,0.782224
Radio,0.054809,1.000000,0.354104,0.576223
Newspaper,0.056648,0.354104,1.000000,0.228299
Sales,0.782224,0.576223,0.228299,1.000000


This relationship increases the variability of the parameter estimates. Thus, collinearity is an issue that needs to be taken care of.

It is a good practice to identify any existing multicollinearity using a correlation matrix, and removing culprit variables before fitting a model. Calculating the **variance inflation factor** (VIF) is a method to tackle this issue.

### Variance Inflation Factor

VIF quantifies increases in the variability of the parameter estimate of a particular variable due to high correlation with one or more other predictor variables. 

The VIF needs to be calculated for each variable *X<sub>i</sub>*. If the value is very high for a particular variable, then that predictor needs to be eliminated from the model. Some statistical processes calculate VIF automatically – as usual, we will briefly discuss the maths behind it first:

1. Write *X<sub>i</sub>* as a linear function of other predictor variables:

   ![](https://latex.codecogs.com/gif.latex?X_i%20%3D%20a_1X_1%20+%20%5Cdots%20+%20a_%7Bi-1%7DX_%7Bi-1%7D%20+%20a_%7Bi+1%7DX_%7Bi+1%7D%20+%20%5Cdots%20+%20a_nX_n)

2. Calculate the coefficient of determination for this model and call it *R<sub>i</sub><sup>2</sup>*. The VIF for *X<sub>i</sub>* is given by:

   ![](https://latex.codecogs.com/gif.latex?%5Ctext%7BVIF%7D%20%3D%20%5Cfrac%7B1%7D%7B1-R_i%5E2%7D)

3. As a rule of thumb, if *VIF > 5*, then multicollinearity is high and the variable needs to be eliminated from the model.

Let’s calculate the VIF for our example to understand the calculation better:

In [9]:
# Initialise and fit model with Newspaper as a linear function of TV and Radio
model = smf.ols('Newspaper ~ TV + Radio', data=advert).fit()

# Calculate VIF
R2 = model.rsquared
VIF = 1 / (1 - R2)
print(f'VIF for Newspaper = {VIF}.')

VIF for Newspaper = 1.1451873787239288.


By changing the formula in the snippet, we can calculate the VIF for the other variables. See if you can do this yourself! 

In [11]:
# Initialise and fit model with TV as a linear function of TV and Radio
model = smf.ols('TV ~ Newspaper + Radio', data=advert).fit()
# Calculate VIF for TV
R2 = model.rsquared
VIF = 1/(1-R2)
print(f'VIF for TV = {np.round(VIF, 4)}')
                
# Initialise and fit model with Radio as a linear function of TV and Newspaper
model = smf.ols('Radio ~ TV + Newspaper', data = advert).fit()

# Calculate VIF for Radio
R2 = model.rsquared
VIF = 1/(1-R2)

print(f'VIF for Radio = {np.round(VIF, 4)}')

VIF for TV = 1.0046
VIF for Radio = 1.145


You should get the following values:

| --- | Newspaper | Radio | TV    |
| --- | --------- | ----- | ----- |
| VIF | 1.145     | 1.145 | 1.004 |

In plain English, this means that `Newspaper` and `Radio` were each inflated by a factor of 1.145 due to correlation with at least one other predictor. We found earlier that they are correlated with each other!

So, what to do? 

The model with `TV` and `Radio` as predictor variables was far better that the model with `TV` and `Newspaper` as the predictor variables, and the model with all the three showed an increase in variability and the F-statistic. 

Thus, it makes sense to drop `Newspaper` from the model and pick model 3 as the best candidate for the final model:

![](https://latex.codecogs.com/gif.latex?%5Ctext%7BSales%7D%3D2.92+0.046*%5Ctext%7BTV%7D+0.188*%5Ctext%7BRadio%7D)

We have learnt how to implement linear regression models and select the best features to increase efficacy of the model. However, so far we've only measured efficacy by comparing the model's predictions with the data used to create the model.

Next, we will discuss how to validate a model by splitting the data into a training and testing set. Go back to the notebook directory in Jupyter by pressing `File` > `Open…` in the toolbar at the top, then open the notebook called `2.4 TestingTraining split.ipynb`.